<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/LanguageTranslationRNN%26LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [3]:
import nltk
nltk.download('punkt')
import LughaatNLP

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from LughaatNLP import LughaatNLP
urdu_text_processing = LughaatNLP()


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [6]:
columns=pd.read_excel('/content/parallel-corpus.xlsx',nrows=0).columns.tolist()
df=pd.read_excel('/content/parallel-corpus.xlsx',usecols=[columns[0],columns[1]])
df.head()
df.describe()

,SENTENCES,MEANING
count,30120,29618
unique,23662,22595
top,Good,اچھی
freq,265,285


#The file iam importing has been cleaned . (Contains stopwords) and its lemmatized to reduce vocabulory size

In [7]:
columns=pd.read_csv('/content/fulldatafullcleaned.csv',nrows=0).columns.tolist()
df=pd.read_csv('/content/fulldatafullcleaned.csv',usecols=[columns[1],columns[2]])
df.describe()
sentences_count_with_nan=len(df)
df.dropna(inplace=True)
sentences_count_without_nan=len(df)
print('Before Nan ',sentences_count_with_nan)
print('After Nan ',sentences_count_without_nan)

df.head()
df.describe()



Before Nan  29617
After Nan  29574


,SENTENCES,MEANING
count,29574,29574
unique,21650,22116
top,good,اچھی
freq,290,290


In [8]:

df = df.drop_duplicates()
df.describe()

,SENTENCES,MEANING
count,22987,22987
unique,21650,22116
top,dont come here,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,8,35


In [9]:
#90 sentences are being covered
threshold = 30
# Count sentences exceeding the threshold
count_exceeding = df[df['SENTENCES '].str.split().str.len() > threshold].shape[0]
print(count_exceeding,len(df))

2501 22987


In [10]:
df = df[df['SENTENCES '].str.split().str.len() <= threshold]


In [11]:
df.describe()

,SENTENCES,MEANING
count,20486,20486
unique,19170,19617
top,dont come here,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,8,35


In [12]:
threshold = 30
# Count sentences exceeding the threshold
count_exceeding = df[df['MEANING'].str.split().str.len() > threshold].shape[0]
print(count_exceeding,len(df))

975 20486


In [13]:
df = df[df['MEANING'].str.split().str.len() <= threshold]


In [14]:


df.describe()

,SENTENCES,MEANING
count,19511,19511
unique,18222,18642
top,dont come here,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,8,35


In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


#useing the tokeinzer on english texts
eng_tokenizer=Tokenizer()
eng_tokenizer.fit_on_texts(df['SENTENCES '])
eng_vocab_size=len(eng_tokenizer.word_index)+1
print('English Vocabulary Size: %d' % eng_vocab_size)
eng_tokenizer.texts_to_sequences(df['SENTENCES '])
padded_eng_sentences=pad_sequences(eng_tokenizer.texts_to_sequences(df['SENTENCES ']),padding='post')


urd_tokenizer=Tokenizer()
urd_tokenizer.fit_on_texts(df['MEANING'])
urd_vocab_size=len(urd_tokenizer.word_index)+1


print('Urdu Vocabulary Size: %d' % urd_vocab_size)
encoded_urdu_sentences=urd_tokenizer.texts_to_sequences(df['MEANING'])
padded_urdu_sentences=pad_sequences(encoded_urdu_sentences,padding='post')

X=padded_eng_sentences
y=padded_urdu_sentences

# Split the dataset into train and test sets (80% train, 20% test)
X_train, X_val, y_train, y_val = train_test_split(padded_eng_sentences, padded_urdu_sentences, test_size=0.1, random_state=42)

# 4. Shifting Target Data
# y_train_shifted = y_train[:, 1:]  # Shift target sequences
# y_val_shifted = y_val[:, 1:]  # Shift target sequences



English Vocabulary Size: 11218
Urdu Vocabulary Size: 11099


#Using the approch of Stacked-Bidirectional LSTM

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# vocab_size=eng_vocab_size+urd_vocab_size
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad your input sequences if they are not already of the same length
max_input_length = 31  # or 30 based on your target
X_train_padded = pad_sequences(X_train, maxlen=max_input_length, padding='post')
X_val_padded = pad_sequences(X_val, maxlen=max_input_length, padding='post')

# If you pad inputs, adjust your target shapes accordingly:
X_train_padded = pad_sequences(y_train, maxlen=max_input_length, padding='post')
y_val_padded = pad_sequences(y_val, maxlen=max_input_length, padding='post')

# Fit the model with the padded sequences
# history = model.fit(X_train_padded, y_train_padded, epochs=10, batch_size=32, validation_data=(X_val_padded, y_val_padded), callbacks=callbacks)


model=tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=eng_vocab_size,output_dim=256,input_length=30),

    # tf.keras.layers.Dense(128,activation='relu'),
        # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)),
  # tf.keras.layers.Dense(256,activation='relu'),
# tf.keras.layers.Dense(256,activation='relu'),

    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(Dense(urd_vocab_size,activation='softmax')) # Added TimeDistributed layer



])




callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)]
model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
history = model.fit(X_train_padded, X_train_padded, epochs=10, batch_size=16, validation_data=(X_val_padded, y_val_padded),callbacks=callbacks)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_18 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_38 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_39 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_13                  │ ?                           │     0 (unbuilt) │
│ (TimeDistributed)                    │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 49s 39ms/step - accuracy: 0.6711 - loss: 2.6458 - val_accuracy: 0.6505 - val_loss: 3.2520
Epoch 2/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 82s 39ms/step - accuracy: 0.8021 - loss: 1.2588 - val_accuracy: 0.6483 - val_loss: 3.8844
Epoch 3/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.8750 - loss: 0.8813 - val_accuracy: 0.6482 - val_loss: 4.2495
Epoch 4/10
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 43s 39ms/step - accuracy: 0.9045 - loss: 0.6769 - val_accuracy: 0.6480 - val_loss: 4.4678


In [33]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (17559, 31)
y_train shape: (17559, 30)
X_val shape: (1952, 31)
y_val shape: (1952, 30)
